In [1]:
# https://www.kaggle.com/code/sarmat/lgbm-stacking-example/notebook

In [2]:
from sklearn import metrics
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import pandas as pd
import numpy as np
import torch
from tqdm import tqdm
from dataset import custom_train_test_split, make_dataset

In [13]:
device = 'cuda'
param1 = {   
            'boosting': 'gbdt',
            'objective': 'binary',
            'num_leaves' : 30,
            'metric' : 'binary_logloss',
            'learning_rate' : 0.1
            
            }

param2 = {   
            'boosting': 'gbdt', 
            # 'max_depth' : 10,
            'objective': 'binary',
            'num_leaves' : 50, # 30 ~ 60
            'metric' : 'binary_logloss',
            'learning_rate' : 0.1 #  0.01~0.3
            }

param3 = {   
            'boosting': 'gbdt',
            'objective': 'binary',
            'num_leaves' : 48,
            'metric' : 'binary_logloss',
            # 'learning_rate' : 0.005
            }

param4 = {   
            'boosting': 'gbdt',
            'objective': 'binary',
            'num_leaves' : 60,
            'metric' : 'binary_logloss',
            # 'learning_rate' : 0.005
            }

param5 = {   
            'boosting': 'gbdt',
            'objective': 'binary',
            'num_leaves' : 70,
            'metric' : 'binary_logloss',
            # 'learning_rate' : 0.01
            }


In [14]:
cat_cols = ['i_head', 'i_mid','i_tail', 'hour', 'dow']
cont_cols = [                        
        'user_correct_answer',
        'user_total_answer',
        'user_acc',            
        't_elapsed',            
        'cum_correct',
        'last_problem',
        'head_term',
        # 'left_asymptote',
        'elo_prob',
        'pkt',
        'u_head_mean',
        'u_head_count',
        'u_head_std',
        'u_head_elapsed',
        'i_mid_elapsed',
        'i_mid_mean',
        'i_mid_std',
        'i_mid_sum',
        'i_mid_count',
        'i_mid_tag_count',
        'assessment_mean',
        'assessment_sum',
        # 'assessment_std',
        'tag_mean',
        'tag_sum',
        # 'tag_std',
        'tail_mean',
        'tail_sum',
        # 'tail_std',
        'hour_mean',
        'hour_sum',
        # 'hour_std',
        'dow_mean',
        'dow_sum',
        # 'dow_std',
        'tag_elapsed',
        'tag_elapsed_o',
        'tag_elapsed_x',
        'assessment_elapsed',
        'assessment_elapsed_o',
        'assessment_elapsed_x',
        'tail_elapsed',
        'tail_elapsed_o',
        'tail_elapsed_x']

FEATS = cat_cols + cont_cols    



In [15]:
train = pd.read_pickle('/opt/ml/level2-dkt-level2-recsys-08/data_pkl/all.pkl')
y = train.answerCode
# train, test = custom_train_test_split(train_data)
# y_train, train, y_test, test = make_dataset(train, test)
test = pd.read_pickle('/opt/ml/level2-dkt-level2-recsys-08/data_pkl/test_data.pkl')
test = test[test.answerCode==-1]

In [16]:
kf = KFold(n_splits=3, random_state = 42,shuffle = True)

oof_clf1 = np.zeros(len(train))
oof_clf2 = np.zeros(len(train))
oof_clf3 = np.zeros(len(train))
oof_clf4 = np.zeros(len(train))
oof_clf5 = np.zeros(len(train))

oof_clf1_t = np.zeros(len(test))
oof_clf2_t = np.zeros(len(test))
oof_clf3_t = np.zeros(len(test))
oof_clf4_t = np.zeros(len(test))
oof_clf5_t = np.zeros(len(test))

for (trn_idx, val_idx),test_val_idx in zip(kf.split(train.values, y.values),kf.split(test.values)):
        test_val_idx = test_val_idx[0]
        # print("fold n°{}".format(fold_))        
        
        lgb_train = lgb.Dataset(train[FEATS].iloc[trn_idx], label = y.iloc[trn_idx]) # lgb.Dataset(train[FEATS], y_train)
        lgb_valid = lgb.Dataset(train[FEATS].iloc[val_idx], label = y.iloc[val_idx]) # lgb.Dataset(valid[FEATS], y_valid)
        
        # num_round? 1000~ 10000
        model1 = lgb.train(param1, lgb_train, 1000, valid_sets=[lgb_valid], early_stopping_rounds=100,verbose_eval=100, categorical_feature = cat_cols)
        model2 = lgb.train(param2, lgb_train, 10000, valid_sets=[lgb_valid], early_stopping_rounds=100,verbose_eval=100, categorical_feature = cat_cols)
        model3 = lgb.train(param3, lgb_train, 10000, valid_sets=[lgb_valid], early_stopping_rounds=100,verbose_eval=100, categorical_feature = cat_cols)
        model4 = lgb.train(param4, lgb_train, 10000, valid_sets=[lgb_valid], early_stopping_rounds=100,verbose_eval=100, categorical_feature = cat_cols)
        model5 = lgb.train(param5, lgb_train, 10000, valid_sets=[lgb_valid], early_stopping_rounds=100,verbose_eval=100, categorical_feature = cat_cols)            
        
        oof_clf1[val_idx] = model1.predict(train[FEATS].iloc[val_idx])
        oof_clf2[val_idx] = model2.predict(train[FEATS].iloc[val_idx])
        oof_clf3[val_idx] = model3.predict(train[FEATS].iloc[val_idx])
        oof_clf4[val_idx] = model4.predict(train[FEATS].iloc[val_idx])
        oof_clf5[val_idx] = model5.predict(train[FEATS].iloc[val_idx])
        
        oof_clf1_t[test_val_idx] = model1.predict(test[FEATS].iloc[test_val_idx])
        oof_clf2_t[test_val_idx] = model2.predict(test[FEATS].iloc[test_val_idx])
        oof_clf3_t[test_val_idx] = model3.predict(test[FEATS].iloc[test_val_idx])
        oof_clf4_t[test_val_idx] = model4.predict(test[FEATS].iloc[test_val_idx])
        oof_clf5_t[test_val_idx] = model5.predict(test[FEATS].iloc[test_val_idx])
        
        # print('Fold no: {}'.format(fold_))
        print("AUC LGB1:{} ".format(metrics.roc_auc_score(y.iloc[val_idx], oof_clf1[val_idx])))
        print("AUC LGB2:{} ".format(metrics.roc_auc_score(y.iloc[val_idx], oof_clf2[val_idx])))
        print("AUC LGB3:{} ".format(metrics.roc_auc_score(y.iloc[val_idx], oof_clf3[val_idx])))
        print("AUC LGB4:{} ".format(metrics.roc_auc_score(y.iloc[val_idx], oof_clf4[val_idx])))
        print("AUC LGB5:{} ".format(metrics.roc_auc_score(y.iloc[val_idx], oof_clf5[val_idx]))) 

/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.8/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['dow', 'hour', 'i_head', 'i_mid', 'i_tail']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 1102675, number of negative: 581295
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.038236 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6623
[LightGBM] [Info] Number of data points in the train set: 1683970, number of used features: 43


/opt/conda/lib/python3.8/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/opt/conda/lib/python3.8/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654807 -> initscore=0.640236
[LightGBM] [Info] Start training from score 0.640236
Training until validation scores don't improve for 100 rounds
[100]	valid_0's binary_logloss: 0.43051
[200]	valid_0's binary_logloss: 0.427988
[300]	valid_0's binary_logloss: 0.427163
[400]	valid_0's binary_logloss: 0.426864
[500]	valid_0's binary_logloss: 0.426688
[600]	valid_0's binary_logloss: 0.426551
[700]	valid_0's binary_logloss: 0.426398
[800]	valid_0's binary_logloss: 0.426277
[900]	valid_0's binary_logloss: 0.426296
[1000]	valid_0's binary_logloss: 0.426253
Did not meet early stopping. Best iteration is:
[974]	valid_0's binary_logloss: 0.426219
[LightGBM] [Info] Number of positive: 1102675, number of negative: 581295
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.041778 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[Lig

/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.8/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['dow', 'hour', 'i_head', 'i_mid', 'i_tail']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


...

[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654337 -> initscore=0.638159
[LightGBM] [Info] Start training from score 0.638159
Training until validation scores don't improve for 100 rounds
[100]	valid_0's binary_logloss: 0.429782
[200]	valid_0's binary_logloss: 0.427361
[300]	valid_0's binary_logloss: 0.426542
[400]	valid_0's binary_logloss: 0.426205
[500]	valid_0's binary_logloss: 0.425999
[600]	valid_0's binary_logloss: 0.425907
[700]	valid_0's binary_logloss: 0.42588
[800]	valid_0's binary_logloss: 0.425796
Early stopping, best iteration is:
[786]	valid_0's binary_logloss: 0.425787
[LightGBM] [Info] Number of positive: 1101885, number of negative: 582086
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.055203 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6622
[LightGBM] [Info] Number of data points in the train set: 1683971,

/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.8/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['dow', 'hour', 'i_head', 'i_mid', 'i_tail']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 1102616, number of negative: 581355
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.046860 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6623
[LightGBM] [Info] Number of data points in the train set: 1683971, number of used features: 43


/opt/conda/lib/python3.8/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/opt/conda/lib/python3.8/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654771 -> initscore=0.640079
[LightGBM] [Info] Start training from score 0.640079
Training until validation scores don't improve for 100 rounds
[100]	valid_0's binary_logloss: 0.430234
[200]	valid_0's binary_logloss: 0.427705
[300]	valid_0's binary_logloss: 0.426869
[400]	valid_0's binary_logloss: 0.426439
[500]	valid_0's binary_logloss: 0.426275
[600]	valid_0's binary_logloss: 0.426104
[700]	valid_0's binary_logloss: 0.425958
[800]	valid_0's binary_logloss: 0.425907
Early stopping, best iteration is:
[778]	valid_0's binary_logloss: 0.425878
[LightGBM] [Info] Number of positive: 1102616, number of negative: 581355
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.041464 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6623
[LightGBM] [Info] Number of data points in the train set: 1683971

In [17]:
Final_train = pd.DataFrame( {
        'LGBM1': oof_clf1.ravel(),
        'LGBM2': oof_clf2.ravel(),
        'LGBM3': oof_clf3.ravel(),
        'LGBM4' :oof_clf4.ravel(),
        'LGBM5': oof_clf5.ravel(),
    })

Final_test = pd.DataFrame( {
        'LGBM1': oof_clf1_t.ravel(),
        'LGBM2': oof_clf2_t.ravel(),
        'LGBM3': oof_clf3_t.ravel(),
        'LGBM4' :oof_clf4_t.ravel(),
        'LGBM5': oof_clf5_t.ravel(),
    })

In [18]:
param = {'objective': 'binary',
                 'boosting': 'gbdt',
                 'random_state': 42,
                 'metric': 'auc',
                 'num_threads': -1,
                 'learning_rate' : 0.1,}

In [19]:
kf = KFold(n_splits=5, random_state = 42,shuffle = True)

Final_oof_train = np.zeros(len(train))
Final_oof_test = np.zeros(len(test))

for (trn_idx, val_idx), test_val_idx in zip(kf.split(Final_train.values, y.values), kf.split(Final_test.values)):
        # print("fold n°{}".format(fold_))
        test_val_idx = test_val_idx[0]
        lgb_train = lgb.Dataset(Final_train.iloc[trn_idx], label = y.iloc[trn_idx])
        lgb_valid = lgb.Dataset(Final_train.iloc[val_idx], label = y.iloc[val_idx])
        
        model = lgb.train(param, lgb_train, 1000, valid_sets=[lgb_valid], early_stopping_rounds=200,verbose_eval=100)
                
        Final_oof_train[val_idx] = model.predict(Final_train.iloc[val_idx])        
        
        Final_oof_test[test_val_idx] = model.predict(Final_test.iloc[test_val_idx])
        
        # print('Fold no: {}'.format(fold_))
        print("AUC LGB1:{} ".format(metrics.roc_auc_score(y.iloc[val_idx], Final_oof_train[val_idx])))

/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 1323032, number of negative: 697732
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009170 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1275
[LightGBM] [Info] Number of data points in the train set: 2020764, number of used features: 5
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654719 -> initscore=0.639846
[LightGBM] [Info] Start training from score 0.639846
Training until validation scores don't improve for 200 rounds
[100]	valid_0's auc: 0.866285
[200]	valid_0's auc: 0.866256
Early stopping, best iteration is:
[48]	valid_0's auc: 0.866288
AUC LGB1:0.8662878705802779 


/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 1322521, number of negative: 698244
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.018642 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1275
[LightGBM] [Info] Number of data points in the train set: 2020765, number of used features: 5
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654466 -> initscore=0.638726
[LightGBM] [Info] Start training from score 0.638726
Training until validation scores don't improve for 200 rounds
[100]	valid_0's auc: 0.866342
[200]	valid_0's auc: 0.866312
Early stopping, best iteration is:
[42]	valid_0's auc: 0.866358
AUC LGB1:0.8663577706457455 


/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 1323005, number of negative: 697760
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.027044 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1275
[LightGBM] [Info] Number of data points in the train set: 2020765, number of used features: 5
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654705 -> initscore=0.639786
[LightGBM] [Info] Start training from score 0.639786
Training until validation scores don't improve for 200 rounds
[100]	valid_0's auc: 0.866262
[200]	valid_0's auc: 0.866221
Early stopping, best iteration is:
[46]	valid_0's auc: 0.866275
AUC LGB1:0.8662754525995435 


/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 1323025, number of negative: 697740
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.010923 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1275
[LightGBM] [Info] Number of data points in the train set: 2020765, number of used features: 5
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654715 -> initscore=0.639830
[LightGBM] [Info] Start training from score 0.639830
Training until validation scores don't improve for 200 rounds
[100]	valid_0's auc: 0.866436
[200]	valid_0's auc: 0.8664
Early stopping, best iteration is:
[50]	valid_0's auc: 0.866447
AUC LGB1:0.866446664580775 


/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 1322769, number of negative: 697996
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.010477 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1275
[LightGBM] [Info] Number of data points in the train set: 2020765, number of used features: 5
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654588 -> initscore=0.639269
[LightGBM] [Info] Start training from score 0.639269
Training until validation scores don't improve for 200 rounds
[100]	valid_0's auc: 0.866603
[200]	valid_0's auc: 0.866576
Early stopping, best iteration is:
[70]	valid_0's auc: 0.866613
AUC LGB1:0.8666129070944879 


In [20]:
Final_oof_test

array([0.46788251, 0.20957201, 0.13674772, 0.91817273, 0.23295179,
       0.81985044, 0.09541078, 0.05498945, 0.12050479, 0.60411493,
       0.13212109, 0.28056417, 0.79540276, 0.07635603, 0.65956372,
       0.91746535, 0.14974235, 0.80022905, 0.92674072, 0.15032902,
       0.90005253, 0.5752921 , 0.37832062, 0.18393309, 0.19942241,
       0.64169827, 0.5649868 , 0.8554898 , 0.32200937, 0.62807214,
       0.67421773, 0.36498613, 0.46744211, 0.06375022, 0.57880483,
       0.9115811 , 0.13301957, 0.2414413 , 0.07672676, 0.05001478,
       0.2145559 , 0.06192513, 0.1900118 , 0.16961278, 0.59006836,
       0.62871669, 0.64087499, 0.196992  , 0.61839725, 0.66717604,
       0.58467226, 0.21089034, 0.20737395, 0.11535902, 0.23322659,
       0.59037103, 0.17229198, 0.89233528, 0.1060587 , 0.0322221 ,
       0.81065495, 0.79896869, 0.91139298, 0.22616395, 0.11442982,
       0.19750195, 0.66131036, 0.23651547, 0.24302509, 0.17739713,
       0.94470951, 0.6437222 , 0.02077158, 0.22564027, 0.65845

In [21]:
def test_to_csv(preds, name:str):
    
    result = []
    for n,i in enumerate(preds):
        row = {}    
        row['id'] = n
        row['prediction'] = i
        result.append(row)
    pd.DataFrame(result).to_csv(f'output/{name}.csv', index=None)

In [22]:
test_to_csv(Final_oof_test,'Ensemble2')